In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sqlalchemy import create_engine
import pyodbc

In [3]:
# Authentication
auth_manager = SpotifyClientCredentials(client_id='id', client_secret='secret')
sp = spotipy.Spotify(auth_manager=auth_manager)

# Create connection to SQL server database
Server = 'localHost'
Database = 'local_db'
Driver = 'ODBC Driver 17 for SQL Server'
Database_Con = f'mssql://@{Server}/{Database}?driver={Driver}'
engine = create_engine(Database_Con)
con = engine.connect()

In [ ]:
# Prompt user to enter a Spotify user and one of their playlists
spotify_playlist_user = input("Please enter a spotify playlist user: ")
spotify_playlist_url = input("Please enter the URL for the user's playlist you want to analyze: ")

In [ ]:
# Split the playlist url to get the playlist code
playlist_url_split_1 = spotify_playlist_url.split("/")
playlist_url_split_2 = playlist_url_split_1[4]
playlist_url_split_2 = playlist_url_split_2.split("?")
playlist_code = playlist_url_split_2[0]

print(playlist_code)

In [ ]:
def get_playlist(creator, playlist_id):
    
    # Create list of features to add as columns to dataframe
    playlist_features_list = ["artist","album","track_name", "track_id",
                              "danceability","energy","key","loudness",
                              "mode", "speechiness","instrumentalness",
                              "liveness","valence","tempo", "duration_ms",
                              "time_signature"]
    
    # Create a playlist dataframe and add catagories for columns
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract data and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dictionary 
        playlist_data = {}
        # Get playlist metadata
        playlist_data["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_data["album"] = track["track"]["album"]["name"]
        playlist_data["track_name"] = track["track"]["name"]
        playlist_data["track_id"] = track["track"]["id"]
        playlist_data["popularity"] = track["track"]["popularity"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_data["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_data[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_data, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
playlist = get_playlist(spotify_playlist_user, playlist_code)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
playlist

In [ ]:
playlist.to_sql('Playlist_Songs_Data', con=engine, if_exists='fail', index=False)